In [3]:
import numpy as np
import pandas as pd 
import polars as pl
import lightgbm as lgb
import joblib

import os

import warnings 
warnings.filterwarnings("ignore")

try:
    import kaggle_evaluation.jane_street_inference_server
except ImportError:
    print("kaggle_evaluation not found.")

kaggle_evaluation not found.


### First Version: Model Training

In [8]:
# total_days = 1699 # Total num of diff date_id = 1699
# valid_days = 120
# lgb_models, _ = train_lgb_holdout_single(total_days=total_days,valid_days=valid_days)

In [9]:
# total_days = 500 # Total num of diff date_id = 1699
# lgb_models, _, _ = train_lgb_kfold_single(total_days=total_days)

### Second Version: Model Loading

In [11]:
# Load the model from the saved file

model_path = '/kaggle/input/jsmodel-chan2'
model_name = 'lgb_model'
models = []
models.append(joblib.load(f'{model_path}/{model_name}.pkl'))

print(f"Loaded model from the saved file.")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/jsmodel-chan2/lgb_model.pkl'

In [ ]:
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    if lags is not None:
        lags_ = lags

    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6')
    )
    
    feat = test[FEAT_COLS + ['weight']].to_pandas()
    # feat = test[FEAT_COLS].to_pandas()

    #model = models[0]
    pred = [model.predict(feat) for model in models]
    #pred = model.predict(feat)
    pred = np.mean(pred, axis=0)
    
    predictions = predictions.with_columns(pl.Series('responder_6', pred.ravel()))
    print(predictions)
    
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)
    
    return predictions

In [ ]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )